In [31]:
import numpy as np 
import pandas as pd
from docplex.mp.model import Model
from docplex.mp.solution import SolveSolution
from typing import Union, List
from numpy.random import default_rng
import names
import pickle

import cplex
import docplex

from datetime import datetime
rng = default_rng()

## Solver Parameters

In [32]:
opt_mod = Model(name='ET MIP')

In [33]:
opt_mod.parameters.timelimit

docplex.mp.params.NumParameter(parameters.timelimit,1e+75)

In [34]:
opt_mod.parameters.timelimit = 10
opt_mod.set_time_limit(10)

## Constants

In [35]:
# number_exams = np.random.randint(10, 20) 
# number_students = np.random.randint(1000, 3000)
# number_exam_days = np.random.randint(14, 15) # number of days allocated for final exams
# number_rooms = np.random.randint(10, 20) 

# E = [f'CSC_{code}' for code in rng.choice(range(101, 401), size=number_exams, replace=False)] # exams
# S = [names.get_full_name() for i in range(number_students)] # students
# T = [[datetime(2022, 12, day, 9), datetime(2022, 12, day, 13), datetime(2022, 4, day, 17)] for day in range(1, number_exam_days)] # timeslots
# R = [f'RM_{code}' for code in rng.choice(range(101, 401), size=number_exams, replace=False)] # rooms
# Cp = np.random.randint(20, 60, (len(R))) # capacity of rooms

# # course enrolments
# He_s = np.random.randint(0,2, (len(E),len(S))) # binary random 
# sumHe_s = np.sum(He_s, axis=1)

In [36]:
# number_exams = 1279
# number_students = 10764
# number_exam_days = 20 #4/11/2022 - 4/31/2022 # number of days allocated for final exams
# number_rooms = 77

# E = [f'APSC_{code}' for code in rng.choice(range(0, number_exams + 1), size=number_exams, replace=False)] # exams
# S = [names.get_full_name() for i in range(number_students)] # students
# T = [[datetime(2022, 12, day, 9), datetime(2022, 12, day, 13), datetime(2022, 12, day, 17)] for day in range(1, number_exam_days)] # timeslots
# R = [f'RM_{code}' for code in rng.choice(range(0, number_rooms + 1), size=number_rooms, replace=False)] # rooms
# Cp = np.random.randint(24, 2000, (len(R))) # capacity of rooms

# # course enrolments
# He_s = np.random.randint(0,2, (len(E),len(S))) # binary random #min:1 max:1948
# sumHe_s = np.sum(He_s, axis=1)

In [37]:
# with open('E_Pickle.pkl', 'wb') as f:
#     pickle.dump(E, f)

# with open('S_Pickle.pkl', 'wb') as f:
#     pickle.dump(S, f)
    
# with open('T_Pickle.pkl', 'wb') as f:
#     pickle.dump(T, f)
    
# with open('R_Pickle.pkl', 'wb') as f:
#     pickle.dump(R, f)

# with open('He_s_Pickle.pkl', 'wb') as f:
#     pickle.dump(He_s, f)


In [38]:
# with open('E_Pickle.pkl', 'rb') as f:
#     E = pickle.load(f)

# with open('S_Pickle.pkl', 'rb') as f:
#     S = pickle.load(f)
    
# with open('T_Pickle.pkl', 'rb') as f:
#     T = pickle.load(f)
    
# with open('R_Pickle.pkl', 'rb') as f:
#     R = pickle.load(f)

# with open('He_s_Pickle.pkl', 'rb') as f:
#    He_s = pickle.load(f)

## Variables

In [39]:
with open('Exam_set_Pickle.pkl', 'rb') as f:
    E = pickle.load(f)

with open('Student_set_Pickle.pkl', 'rb') as f:
    S = pickle.load(f)
    
with open('Timeslots_Pickle.pkl', 'rb') as f:
    T = pickle.load(f)
    
with open('Rooms_Pickle.pkl', 'rb') as f:
    R = pickle.load(f)

with open('course enrolments_Pickle.pkl', 'rb') as f:
   He_s = pickle.load(f)
   
with open('Room_Capacity.pkl', 'rb') as f:
    Cp =pickle.load(f)

In [40]:
sumHe_s = np.sum(He_s, axis=1)

In [41]:
x = opt_mod.binary_var_matrix(len(E), len(T), name="X_e,t") # whether we use timeslot t for exam e
y = opt_mod.binary_var_matrix(len(E), len(R), name="Y_e,r") # whether we use room r for exam e
# z = opt_mod.binary_var_matrix(len(S), len(E), name="Z_s,e") # whether exam e is allocated to student s 

## Constraints

C1: For all exams, the sum of the allocated timeslots must be equal to 1

$$\sum_{t\in T_c} X_e,_t=1 \;\forall \; e \in E$$

C2: For all exams, the sum of the allocated rooms must be equal to 1

$$\sum_{r\in R_e} Y_e,_r = 1 \;\forall \; e \in E$$

C3: For every student and timeslot, the sum of the allocated exams must be less or equal to 1. 
- i.e. students can be at only one exam at a time

$$\sum_{e\in E} X_e,_t * H_e,_s \leq 1 \;\forall \; s \in S \; and \; t \in T$$


C4: For all rooms, the sum of students in a room must be less than the capacity of the room

$$\sum_{e\in E} X_e,_t * y_e,_r \leq C_p,_r \;\forall \; r \in R \; and \; t \in T$$

In [79]:
c1 = opt_mod.add_constraints((sum(x[e, t] for t in range(len(T))) == 1 for e in range(len(E))), names='c1') 

In [80]:
c2 = opt_mod.add_constraints((sum(y[e, r] for r in range(len(R))) >= 1 for e in range(len(E))), names='c2') 

In [81]:
# c3 modified constraint 
for s in range(len(S)):
    for t in range(len(T)):
        cond = sum(x[e,t] * He_s[e,s] for e in range(len(E)))
        if type(cond) != int:
            opt_mod.add_constraint(cond <= 1)

In [82]:
# c4 modified constraint
for r in range(len(R)):
    for t in range(len(T)):
        cond = sum((x[e,t]*y[e,r]) * sumHe_s[e] for e in range(len(E)))
        if type(cond) != int:
            opt_mod.add_constraint(cond <= Cp[r])

In [83]:
# c4 = opt_mod.add_constraints((sum(y[e,r] * sumHe_s[e] for e in range(len(E))) <= Cp[r] for r in range(len(R))), names='c4') 

## Objective Function

$$  minimize\; I_T = \sum_{k=1}^{K_T} \; ceil \; \left[ \sum_{c=1}^{C_K}\; N_c \; * \; (ratio \; students \; to \; invigilators) \right] $$

In [84]:
ratio_of_Inv = 1/50

$$\sum_{r \in R} \; ceil \;  \left[ \sum_{e \in E}y_e,_r ( \sum_{s \in S} H_e,_s ) \; (ratio \; students \; to \; invigilators) \right] * \; cost \; per \; invigilator $$

In [85]:
up = (sum(1 * sumHe_s[e] * ratio_of_Inv for e in range(len(E))) for r in range(len(R)))
upper_bound=0
for i in up:
    upper_bound += np.ceil(i)
#print(upper_bound)

ceil_obj = []
OB = []
var=[]
qwe = []
sum_sum = []

for r in range(len(R)):
    ceil_obj.append(opt_mod.integer_var(lb=0, ub= upper_bound))
    sum_sum.append(sum(y[e,r] * sumHe_s[e] * ratio_of_Inv for e in range(len(E))))
    opt_mod.add_constraint(ceil_obj[r] >= sum_sum[r])


In [86]:
OB = sum(ceil_obj[r] for r in range(len(R)))
opt_mod.set_objective('min', OB)
opt_mod.print_information()

Model: ET MIP
 - number of variables: 21733
   - binary=21218, integer=515, continuous=0
 - number of constraints: 124939
   - linear=103721, quadratic=21218
 - parameters:
     parameters.timelimit = 10.00000000000000
 - objective: minimize
 - problem type is: MIQCP


In [87]:
# obj_fun = sum(sum(y[e,r] * sumHe_s[e] *ratio_of_Inv for e in range(len(E))) for r in range(len(R)))
# opt_mod.set_objective('min', obj_fun)
# opt_mod.print_information()

## Processing Solution

In [88]:
def process_solution(sol : SolveSolution) -> Union[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Takes a cplex solution and produces a exam schedule
    
    Parameters
    ----------
    sol : SolveSolution
        solution from the solver
    
    Returns
    -------
    final_schedule : pd.DataFrame
        The schedule formatted in readable format for an exam organizer
    
    df_x : pd.DataFrame
        The results for variable x
    
    df_y : pd.DataFrame
        The results for variable y
    """
    # extract solutions as df
    df_x = sol.get_value_df(x).rename(columns={'key_1':'exam','key_2':'timeslot'})
    df_y = sol.get_value_df(y).rename(columns={'key_1':'exam','key_2':'room'})

    # Add rows with the names of courses and timelots
    exam_col = [E[i] for i in range(len(E)) for j in range(len(T))]
    time_col = [T[j] for i in range(len(E)) for j in range(len(T))]
    df_x["EXAM"] = exam_col
    df_x["TIMESLOT"] = time_col

    # Add rows with the names of courses and rooms
    exam_col = [E[i] for i in range(len(E)) for j in range(len(R))]
    room_col = [R[j] for i in range(len(E)) for j in range(len(R))]
    df_y["EXAM"] = exam_col
    df_y["ROOM"] = room_col
    
    # Produce the final schedule
    final_schedule = df_x[df_x["value"]==1].merge(df_y[df_y["value"]==1], on='EXAM', how='left')
    
    return final_schedule, df_x, df_y

In [89]:
def create_enrolment_df(He_s : np.array, S : List[int]) -> pd.DataFrame:
    """
    Creates a dataframe with the students for each exam/course
    """
    exam_student_pairs = []
    for exam in range(len(He_s)):
        students_in_exam_e = []
        for i, student in enumerate(He_s[exam]):
            if student == 1:
                students_in_exam_e.append(S[i])
        exam_student_pairs.append(students_in_exam_e)
        
    enrolment_df = pd.DataFrame(columns=['EXAM','student'])
    enrolment_df['EXAM'] = E
    enrolment_df['student'] = exam_student_pairs

    return enrolment_df

In [90]:
enrolment_df = create_enrolment_df(He_s, S)
enrolment_df

,EXAM,student
0,AER302H1S,[0x15698616B7B1051E1D402B152BCC092ABA084917]
1,AER372H1S,"[0x079994DE6FEDC000090ECA1859B31D1A68F0037C, 0..."
2,APS105H1S,[0x121AC60485E5046D3A77059C540885F0312BE38D]
3,APS105H1S,[0x0557E59C7FF9F3A35DA2C225479C382504D6F214]
4,APS106H1S,[0x10F44D0164CE98467C77987548ADE68DF73C246B]
...,...,...
98,CSC401H1S,"[0x0319B30530028A29C9839174748616989D664D2E, 0..."
99,CSC488H1S,"[0x0E4086D5C02D028399AAC080935F1A12BD24DBBE, 0..."
100,GGR124H1S,"[0x12965E9F694E8A825987895DE15209FA7DEC02DB, 0..."
101,GGR124H1S,"[0x06B35E1ACE848948DF99D2DB30007C61AE80B23E, 0..."


In [91]:
sol = opt_mod.solve()
if sol:
    print("Found a solution \n")
    schedule, df_x, df_y = process_solution(sol)
    print("Schedule: \n")
    display(schedule.merge(enrolment_df, on='EXAM', how='left'))
    
    run_time = sol.solve_details.time
    
else:
    print("Could not find a solution")
    run_time = np.nan

Could not find a solution
